In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import time
import os




In [3]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))


web = "https://twitter.com/i/flow/login"
driver.get(web)

# wait of 6 seconds to let the page load the content
time.sleep(6)  # this time might vary depending the network

twitter_email = ''
twitter_username = ''
twitter_password = ''

username = driver.find_element(By.XPATH,'//input[@autocomplete ="username"]')
username.send_keys(twitter_email)



# Clicking on "Next" button
next_button = driver.find_element(By.XPATH,'//div[@role="button"]//span[text()="Next"]')
next_button.click()
time.sleep(5)

# anti_check = driver.find_element(By.XPATH,'//input[@autocapitalize="none"]')
# anti_check.send_keys(twitter_username)

# # Clicking on "Next" button
# next_button = driver.find_element(By.XPATH,'//div[@role="button"]//span[text()="Next"]')
# next_button.click()
# time.sleep(5)


# locate password textbox
password = driver.find_element(By.XPATH,'//input[@autocomplete ="current-password"]')
password.send_keys(twitter_password) 


# locating login button and then clicking on it
login_button = driver.find_element(By.XPATH, '//div[@role="button"]//span[text()="Log in"]')
login_button.click()

In [5]:
topic = 'Python'
search_button = driver.find_element(By.XPATH,'//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div/div/form/div[1]/div/div/div/label/div[2]/div/input')
search_button.send_keys(topic)
search_button.send_keys(Keys.ENTER)


In [21]:
# print(driver.page_source)
# soup = BeautifulSoup(driver.page_source, 'lxml')
# postings = soup.find_all('div', class_="css-901oao r-18jsvk2 r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-bnwqim r-qvutc0")
# tweets = []

# while True:
#     for post in postings:
#         tweets.append(post.text)
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     time.sleep(3)
#     soup = BeautifulSoup(driver.page_source, 'lxml')
#     postings = soup.find_all('div', class_="css-901oao r-18jsvk2 r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-bnwqim r-qvutc0")
#     tweets2 = list(set(tweets))
#     if len(tweets2) > 200:
#         break

def get_tweet(element):
    """This function scrapes data of tweets. It returns a list with 2 elements; username and text"""
    try:
        user = element.find_element(By.XPATH, ".//span[contains(text(), '@')]").text
        text = element.find_element(By.XPATH, ".//div[@lang]").text
        tweets_data = [user, text]
    except:
        tweets_data = ['user', 'text']
    return tweets_data

user_data = []
text_data = []
tweet_ids = set()
scrolling = True
while scrolling:
    tweets = WebDriverWait(driver, 5).until(
        EC.presence_of_all_elements_located((By.XPATH, "//article[@role='article']")))
    print(len(tweets))
    # you can change this number with the number of tweets in a website || 
    # NOTE: ONLY THOSE LOADED IN THE last page will be considered while those from previous page will be forgotten (example: scroll all the way down and then try to find an @username that it's on top --> it won't find it)
    for tweet in tweets[-15:]: 
        tweet_list = get_tweet(tweet)
        tweet_id = ''.join(tweet_list)
        if tweet_id not in tweet_ids:
            tweet_ids.add(tweet_id)
            user_data.append(tweet_list[0])
            text_data.append(" ".join(tweet_list[1].split()))

    # Get the initial scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(3)
        # Calculate new scroll height and compare it with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        # break according to the height is not work now 
#         if new_height == last_height:  # if the new and last height are equal, it means that there isn't any new page to load, so we stop scrolling
#             print('height reached')
#             scrolling = False
#             break
#       break condition based on the number of rows we get works.
        if len(user_data) > 100: #get 100 data
            print('length reached')
            scrolling = False
            break
        else:
            last_height = new_height
            break


# driver.quit()
print(scrolling)
df_tweets = pd.DataFrame({'user': user_data, 'text': text_data})
df_tweets.to_csv('tweets.csv', index=False)
print(df_tweets)



7
7
7
7
9
12
12
5
10
9
11
11
11
11
11
11
11
11
11
11
11
3
12
9
9
9
5
13
13
13
6
length reached
False
                user                                               text
0    @SwissCognitive  Required #DataScience skills By: coders learni...
1      @Jensen_Hacks  Useful roadmap for #Python #MachineLearning #D...
2            @tut_ml  Learn Data Science with these FREE Online Cour...
3     @ezekiel_aleke  Python SQL PowerBI Excel Statistics Tableau and I
4     @python_spaces  If you want to get started with Python, check ...
..               ...                                                ...
99      @gmrpetricca  2023-03-08 M5.8 #Kuril Islands #earthquake rec...
100  @Mountaingoat55  I find it hard to understand how 2 anti extabl...
101     @Newcoupon01  Udemy Free Discount - Write PHP Like a Pro: Bu...
102       @OpShinDev  Introducing: hebi Part of the op-shin (#eopsin...
103      @driscollis  What is your favorite science-related #Python ...

[104 rows x 2 columns]


In [17]:


# web = "https://twitter.com/search?q=python&src=typed_query"
# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# driver.get(web)
# driver.maximize_window()

# user_data = []
# text_data = []

# def get_tweet(element):
#     """This function scrapes data of tweets. It returns a list with 2 elements; username and text"""
#     try:
#         user = element.find_element(By.XPATH, ".//span[contains(text(), '@')]").text
#         text = element.find_element(By.XPATH, ".//div[@lang]").text
#         tweets_data = [user, text]
#     except:
#         tweets_data = ['user', 'text']
#     return tweets_data


# # Getting all the tweet cards/boxes listed in a single page
# tweets = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, "//article[@role='article']")))


# for tweet in tweets:
#     tweet_list = get_tweet(tweet)  # calling the function get_tweet to scrape data of the tweets list
#     user_data.append(tweet_list[0])  # appending the first element of tweet_list (user)
#     text_data.append(" ".join(tweet_list[1].split()))  # appending the second element of tweet_list (text)
    
    

# df_tweets = pd.DataFrame({'user': user_data, 'text': text_data})
# df_tweets.to_csv('tweets.csv', index=False)
# print(df_tweets)

               user                                               text
0          @greggyb  Built a command line chatbot in Python with th...
1  @Fabriziobustama  Mixed reality play space blending physical & v...
2        @buildkite  Deploy at scale with our free Developer Plan. ...
3    @SecurityTrybe  Cybersecurity Roadmap 2023 #CyberSecurityAware...


In [25]:

# web='https://twitter.com/CommunityNotes/status/1628158167006994436?cxt=HHwWiIDT6b_OsJgtAAAA'

# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# driver.get(web)
# driver.maximize_window()

# last_height = driver.execute_script("return document.body.scrollHeight")
# while True:
#     # Scroll down to bottom
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     # Wait to load page
#     time.sleep(5)
#     # Calculate new scroll height and compare it with last scroll height
#     new_height = driver.execute_script("return document.body.scrollHeight")
#     if new_height == last_height:  # if the new and last height are equal, it means that there isn't any new page to load, so we stop scrolling
#         break
#     else:
#         last_height = new_height

In [24]:
# web='https://twitter.com/CommunityNotes/status/1628158167006994436?cxt=HHwWiIDT6b_OsJgtAAAA'

# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# driver.get(web)
# driver.maximize_window()

# last_height = driver.execute_script("return document.body.scrollHeight")
# scroll_count = 0
# while scroll_count < 10:
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     time.sleep(5)
#     new_height = driver.execute_script("return document.body.scrollHeight")
#     if new_height == last_height:
#         break
#     last_height = new_height
#     scroll_count += 1